In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

This non-commercial license of GraphLab Create for academic use is assigned to sachinjain024@gmail.com and will expire on August 18, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1504681634.log


## Create k-nearest neighbour model using deep features

In [3]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


In [4]:
# label inside knn model is the column against which results will be indexed.
knn_model = graphlab.nearest_neighbors.create(image_train, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [13]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [10]:
knn_model.query(image_train[18:19])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 10.093ms     |

| Done         |         | 100         | 116.533ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


### We need to see the images of the returned result so we will define a custom function to get the image from reference_label which is actually the id of the image row

In [11]:
def get_images_from_id(query_result):
    return image_train.filter_by(query_result['reference_label'], 'id')

In [14]:
cat_neighbours = get_images_from_id(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.332ms      |

| Done         |         | 100         | 109.18ms     |

+--------------+---------+-------------+--------------+

In [15]:
cat_neighbours['image'].show()

### Find nearest neibhours for car

In [16]:
car = image_train[8:9]
car['image'].show()

In [17]:
get_images_from_id(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.694ms      |

| Done         |         | 100         | 126.635ms    |

+--------------+---------+-------------+--------------+

## Using lambda function to get neighbours

In [18]:
show_neighbours = lambda index: get_images_from_id(knn_model.query(image_train[index: index + 1]))['image'].show()

In [19]:
show_neighbours(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.019ms      |

| Done         |         | 100         | 125.77ms     |

+--------------+---------+-------------+--------------+

In [20]:
show_neighbours(18)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.411ms      |

| Done         |         | 100         | 133.607ms    |

+--------------+---------+-------------+--------------+

In [21]:
show_neighbours(1000)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.138ms      |

| Done         |         | 100         | 121.462ms    |

+--------------+---------+-------------+--------------+

In [22]:
show_neighbours(2000)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.981ms      |

| Done         |         | 100         | 114.58ms     |

+--------------+---------+-------------+--------------+

In [24]:
show_neighbours(1500)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.379ms      |

| Done         |         | 100         | 124.35ms     |

+--------------+---------+-------------+--------------+

## Assignment

### Question1: Computing summary statistics of the data:

In [26]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


### Question2: Creating category-specific image retrieval models

In [27]:
cat_data = image_train[image_train['label'] == 'cat']
dog_data = image_train[image_train['label'] == 'dog']
bird_data = image_train[image_train['label'] == 'bird']
automobile_data = image_train[image_train['label'] == 'automobile']

In [28]:
cat_model = graphlab.nearest_neighbors.create(cat_data, features=['deep_features'], label='id')
dog_model = graphlab.nearest_neighbors.create(dog_data, features=['deep_features'], label='id')
bird_model = graphlab.nearest_neighbors.create(bird_data, features=['deep_features'], label='id')
automobile_model = graphlab.nearest_neighbors.create(automobile_data, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [78]:
result = cat_model.query(image_test[0:1])
image_test[0:1]['image'].show()
get_images_from_id(result)['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.64ms       |

| Done         |         | 100         | 44.214ms     |

+--------------+---------+-------------+--------------+

In [79]:
dog_result = dog_model.query(image_test[0:1])
get_images_from_id(dog_result)['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.243ms      |

| Done         |         | 100         | 46.243ms     |

+--------------+---------+-------------+--------------+

In [33]:
get_images_from_id(dog_result)['image'].show()

## Question3

### For the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘cat’ in the training data

In [35]:
result = cat_model.query(image_test[0:1])
print result

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.751ms      |

| Done         |         | 100         | 44.136ms     |

+--------------+---------+-------------+--------------+

+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]



In [36]:
result['distance'].mean()

36.15573070978294

### Similarly, for the first image in the test data (image_test[0:1]), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘dog’

In [38]:
result = dog_model.query(image_test[0:1])
print result
result['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.408ms      |

| Done         |         | 100         | 42.905ms     |

+--------------+---------+-------------+--------------+

+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]



37.77071136184157

### On average, is the first image in the test data closer to its 5 nearest neighbors in the ‘cat’ data or in the ‘dog’ data? (In a later course, we will see that this is an example of what is called a k-nearest neighbors classifier, where we use the label of neighboring points to predict the label of a test point.)


## Question 4: Computing nearest neighbors accuracy using SFrame operations

In [39]:
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_bird = image_test[image_test['label'] == 'bird']
image_test_automobile = image_test[image_test['label'] == 'automobile']

In [52]:
# Note that the input index will be stored in the ‘query_label’ column of the output SFrame.
test_dog_neighbour_to_train_cat = cat_model.query(image_test_dog, k=1)
test_dog_neighbour_to_train_dog = dog_model.query(image_test_dog, k=1)
test_dog_neighbour_to_train_bird = bird_model.query(image_test_dog, k=1)
test_dog_neighbour_to_train_automobile = automobile_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 266.032ms    |

| Done         | 509000  | 100         | 288.389ms    |

+--------------+---------+-------------+--------------+

In [57]:
## Creating a new sFrame 
dog_distances = graphlab.SFrame({
    'dog_cat': test_dog_neighbour_to_train_cat['distance'],
    'dog_dog': test_dog_neighbour_to_train_dog['distance'],
    'dog_bird': test_dog_neighbour_to_train_bird['distance'],
    'dog_automobile': test_dog_neighbour_to_train_automobile['distance']
})

dog_distances.head()

dog_automobile,dog_bird,dog_cat,dog_dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373
46.0021331807,41.3382958925,38.8353268874,32.8458495684
42.9462290692,38.6157590853,36.9763410854,35.0397073189
41.6866060048,37.0892269954,34.5750072914,33.9010327697
39.2269664935,38.272288694,34.778824791,37.4849250909
40.5845117698,39.1462089236,35.1171578292,34.945165344
45.1067352961,40.523040106,40.6095830913,39.0957278345
41.3221140974,38.1947918393,39.9036867306,37.7696131032
41.8244654995,40.1567131661,38.0674700168,35.1089144603
45.4976929401,45.5597962603,42.7258732951,43.2422832585


### Compute the number of ‘dog’ test examples where the distance to the nearest ‘dog’ was lower than that to the other classes.

In [63]:
def is_dog_correct(row):
    if row['dog_dog'] < row['dog_cat'] and row['dog_dog'] < row['dog_bird'] and row['dog_dog'] < row['dog_automobile']:
        return 1
    else:
        return 0

In [68]:
dog_distances.apply(is_dog_correct).sum()

678

### Accuracy of predicting dog in the test data

In [69]:
len(dog_distances)

1000

In [70]:
len(dog_data)

509

In [71]:
len(image_test_dog)

1000